# Sparse Optimization for Classification Example

Based on the work of Brunton, et al[<sup>1</sup>](#fn1)

## Outline

The strategy to be employed is as follows:
1. Project the data to an ideal basis. Here we will use principal components analysis (PCA).
2. Identify a maximally-discriminating projection vector to the new subspace.  Here we use linear discriminant analysis (LDA).
3. Find the sparsest vector in the original feature space that maps to the maximally-discriminating projection vector.  Here we use convex constrained optimization.
4. Use the sparse vector to create a measurement matrix that sparsifies the images.
5. Train a classifier on the sparsified data.  Here we use logistic regression.

## 1. Prepare the data
Load the packages required for the example.  This may take some time, as Flux requires a good deal of precompilation.

In [ ]:
using DelimitedFiles, LinearAlgebra, Statistics, Convex, COSMO, StatsBase, Flux
using Flux.Data:DataLoader
using Flux.Losses: logitbinarycrossentropy
using Flux: onecold, onehotbatch
using StatsBase, PyCall

const DIR = @__DIR__

Create a pyplot object for plotting.

In [ ]:
plt = pyimport("matplotlib.pyplot");

### Load the datasets for the example.  

These are vectorized 64x64 pixel grayscale images of cats and dogs in column-major format (arranged as a matrix with pixels as rows and images as columns).  There are 80 images of each type, for a total of 160 images.

In [ ]:
cats = Float32.(readdlm(DIR*"/SSPOC/data/cats.csv", ','))
dogs = Float32.(readdlm(DIR*"/SSPOC/data/dogs.csv", ','))

Randomize the images within classes in order to account for any bias in the original ordering.

In [ ]:
cats = cats[:, sample(1:size(cats, 2), size(cats, 2), replace=false)]
dogs = dogs[:, sample(1:size(dogs, 2), size(dogs, 2), replace=false)]

View a few example images.

In [ ]:
fig, ax = plt.subplots(2,3)
cmap = plt.cm.gray
for i = 1:2:5
    ax[i].imshow(reshape(cats[:,i], (64, 64)), cmap=cmap)
end
for i = 2:2:6
    ax[i].imshow(reshape(dogs[:,i], (64, 64)), cmap=cmap)
end
plt.show()

### Create training and testing sets.  

The first 60 images of each type will be used for training.  The remaining 20 of each type will be for testing.

In [ ]:
train1 = [cats[:,1:60] dogs[:,1:60]];
test1 = [cats[:,61:end] dogs[:,61:end]];

## 2. Get the eigenrepresentation
Compute the PCA to get the eigenrepresentation.  The left singular vectors now form an orthonormal basis for the column space of the data.  This means that the new coordinate system can be used to encode correlation between images.  Conversely, the right singular vectors form an orthonormal basis for the row space, which is not what we want to use here (you can see this for yourself by projecting the original data onto the right singular subspace).

In [ ]:
n, m = size(train1)
## centering matrix
C = I(m) - (1/m) * ones(m,m)
train_centered = train1 * C
U, λ, V = svd(train_centered)

The loadings of each feature onto individual images is given by the $\mathbf{V}$ (right singular vectors) matrix.  The loadings for the first 3 modes of the $\mathbf{V}$ matrix are shown for each image type.  It can be seen that certain modes capture more separability than others.

In [ ]:
xbin = -.25:.05:.25
fig, ax = plt.subplots(1,3)
for i = 1:3
    ax[i].hist(V[1:80,i], color="red", bins=xbin, label="cats")
    ax[i].hist(V[81:end, i], color="blue", bins=xbin, label="dogs")
end
plt.legend()
plt.show()

We can plot the eigenvalues to identify a reasonable reduced rank in a principled, albeit unrigorous, way.  The idea is to use a minimal-rank representation that still adequately encodes the data.  Our assumption that many complex phenomena exhibit low-rank structure is depicted graphically by the elbow shape of the plot.  This means that, in this case, beyond around the first 30 to 40 eigenvalues, the remaining 80 to 90 contribute (relatively) little to the structure, and can be truncated with minimal loss of information.  Mathematically rigorous methods for identifying an optimal rank exist, but are beyond the scope of this example.

In [ ]:
plt.plot(1:length(λ), λ)
plt.xlabel("Eigenvalue index")
plt.ylabel("Value")
plt.show()

We can now choose a reduced-rank basis Ψ for the subspace (here we somewhat arbitrarily choose 40).  Recall that the left singular matrix, $\mathbf{U}$, captures the correlations between images.

In [ ]:
Ψ = U[:,1:40]

The modes of this space represent the dominant features across images.  We can view these by reshaping the columns of $\Psi$.  The pictures of the 6 leading eigenfaces suggest that the shape of cats' ears are highly important.  This is intuitive, as dogs generally exhibit a more diverse set of phenotpyes than cats (cats normally never have floppy ears).

In [ ]:
fig, ax = plt.subplots(2,3)
cmap = plt.cm.gray
for i = 1:2:5
    ax[i].imshow(reshape(Ψ[:,i], (64, 64)), cmap=cmap)
end
for i = 2:2:6
    ax[i].imshow(reshape(Ψ[:,i], (64, 64)), cmap=cmap)
end
plt.show()

Projecting the original data onto the new space will transform the images into their fundamental representation, $\mathbf{\tilde{X}}$.

In [ ]:
X̃ = Ψ' * train1

## 3. Compute the maximally discriminating vector using LDA
We can use linear discriminant analysis (LDA) to find a vector that maximally discriminates between the two image classes.  Really, any statistical method can be used here, but LDA is straightforward, interpretable, and very effective for this problem.

### Compute between-classes scatter matrix
Since there are only two classes and they are balanced, the between-class scatter matrix is given by $\mathbf{S_B} = (\mathbf{\mu}_1 - \mathbf{\mu_2})(\mathbf{\mu}_1 - \mathbf{\mu}_2)^T$.

In [ ]:
μ1 = mean(X̃[:, 1:60], dims=2)
μ2 = mean(X̃[:,61:end], dims=2)
Sb = (μ2 - μ1) * (μ2 - μ1)'

### Compute the within-classes scatter matrix
The within-class scatter is given by $\mathbf{S_W} = (\mathbf{X_1} - \mathbf{\mu}_1)(\mathbf{X_2} - \mathbf{\mu}_2)^T$.

In [ ]:
function scatter_matrix(data, mu)
    (data - mu .* ones(size(data))) * (data - mu .* ones(size(data)))'
end
## within-classes scatter Sw
Σ1 = scatter_matrix(X̃[:,1:60], μ1)
Σ2 = scatter_matrix(X̃[:,61:end], μ2)
Sw = Σ1 + Σ2

### Compute the projection matrix and get the maximum-discriminating projection vector

We want to find the projection that maximizes the between-class scatter, $\mathbf{S_B}$, with respect to the within-class scatter, $\mathbf{S_W}$.  The Rayleigh quotient

$$\hat{\mathbf{u}} =\underset{\mathbf{u}}{\operatorname{arg max}}\dfrac{\mathbf{u}^T\mathbf{S_Bu}}{\mathbf{u}^T\mathbf{S_Wu}}$$

can be solved as a generalized eigenvalue problem

$$\mathbf{S_B}\mathbf{U} = \mathbf{\Lambda} \mathbf{S_WU}$$

In [ ]:
Λ, W = eigen(Sb, Sw)

Choose eigenvector $\mathbf{w}$ corresponding to the leading eigenvalue (this represents the maximally discriminating projection vector in the new subspace).

In [ ]:
Λ = reverse(Λ)
W = reverse(W, dims=2)
w = real(W[:,1])

## 4. Find a sparse vector in original space that maps to $\mathbf{w}$

$\mathbf{s}$ (the maximally sparse vector in original space that maps to $\mathbf{w}$) can be found by solving the optimizaton problem

$$\hat{\mathbf{s}} =\underset{\mathbf{s}}{\operatorname{arg min}}\lvert\lvert \mathbf{s}\rvert\rvert_0 {\text{  subject to  }} \mathbf{w} = \Psi \mathbf{s}$$
    
where $\lvert\lvert \cdot \rvert\rvert_0$ is the cardinality.
This can be relaxed to 

$$\hat{\mathbf{s}} =\underset{\mathbf{s}}{\operatorname{arg min}}\lvert\lvert \mathbf{s}\rvert\rvert_1 {\text{  subject to  }} \mathbf{w} = \Psi \mathbf{s}$$

provided that the nonzero elements of $\mathbf{s}$ are incoherent with respect to $\Psi$.  This is a reasonble assumption since the nonzero elements correspond to pixels in the original space.  However, the L1 relaxation of the objective function is a probabilistic assumption.  We can say there is a high probability of it holding, but there is no mathematical guarantee.

When using the solver, note that a good approximation can still be achieved without convergence.


In [ ]:
n = size(train1, 1)
s = Variable(n)
objective = norm(s,1)
constraint = Ψ' * s == w;
solver = COSMO.Optimizer;
problem = minimize(objective, constraint);
solve!(problem, solver);

Check linear separability imposed by sparse vector $\hat{\mathbf{s}}$.  The $y$-axis represents the loadings and images are arranged along the $x$-axis (1-60 are cats and 61-120 are dogs).  The discrimination function lies at $y = 0$, so the values for cats and dogs should be mostly of opposite sign.  Results may vary across runs, which highlights the importance of cross-validation in actual practice (recall the above statement concerning the probabalistic nature of the underlying assumptions). 

In [ ]:
ŝ = s.value[:]
sep = (Ψ' * ŝ)' * (Ψ' * train1)
plt.bar(1:length(sep), sep)
plt.xlabel("Images (cats=1-60, dogs=61-120)")
plt.ylabel("Loadings")
plt.show()
StatsBase.describe(ŝ)

Get the 20 pixels with the largest value.  This is an arbitrary decision made for demonstration purposes.  A principled approach would use cross-validation to identify an optimal value.  Here, the purpose is to demonstrate that a very small portion of the original feature space can produce relatively good results on a statistical learning task.

In [ ]:
thresh = abs(ŝ[reverse(sortperm(abs.(ŝ)))[21]])
C = abs.(ŝ) .> thresh

Verify how many pixels will be used.

In [ ]:
sum(C)

This means that less than 0.5% of the original pixels will be used for the classification task!

## 5. Train a logistic regression classifier to use the sparse image

Create a measurement matrix, $\Phi$, by creating a square matrix with the desired pixel indices on the diagonal.  This will serve to "select" the maximally-discriminating pixels we care about for the classification task.

In [ ]:
Φ = Diagonal(C)

Scale data with z score transform to improve the stability and performance of the model (since the pixel values range from 0 to 255, there are no extreme outliers requiring more advanced techniques).

In [ ]:
dt = StatsBase.fit(ZScoreTransform, train1, dims=2)
StatsBase.transform!(dt, train1)

Sparsify the original data with the measurement matrix, using the measurement matrix to send all but the imprtant pixels to zero.

In [ ]:
y = Φ * train1

Show which pixels will be used in a few example images. The third image is of the leading eigenface, to provide context as to the space which was used in determining pixel location.  Notice pixels capture certain features of the animals' faces. 

In [ ]:
cat_sparse_pixels = deepcopy(cats[:, 36])
dog_sparse_pixels = deepcopy(dogs[:, 36])
eig_sparse_pixels = deepcopy(Ψ[:,1])
cat_sparse_pixels[C] .= NaN
eig_sparse_pixels[C] .= NaN
dog_sparse_pixels[C] .= NaN
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
cmap = plt.cm.gray
cmap.set_bad((1, 0, 0, 1))
ax1.imshow(reshape(cat_sparse_pixels, (64,64)), cmap=cmap)
ax3.imshow(reshape(eig_sparse_pixels, (64,64)), cmap=cmap)
ax2.imshow(reshape(dog_sparse_pixels, (64,64)), cmap=cmap)
plt.show()

Create labels for the images and prepare the data for training.

In [ ]:
labels = Int.(vcat(ones(60), zeros(60)))
l = onehotbatch(labels, [0, 1])
train = DataLoader((y, l), batchsize=32);

Build a logistic regression model for the binary classification task.  In principle, any classification method can be used here.  Logistic regression was chosen for its simplicity, given that there are only 2 classes.  Also note that since sparsifying the data with the measurement matrix selects features similar to L1 regularization, no explicit regularizer will be used here.

In [ ]:
function loss(x, y)
    logitbinarycrossentropy(m(x), y)
end
opt = ADADelta(.5)
m = Dense(4096, 2)
# burn-in model
m(rand(Float32, 4096))

Train for 100 epochs of gradient descent.

In [ ]:
@info "Starting training..."
for epoch in 1:100
    Flux.train!(loss, params(m), train, opt)
    if mod(epoch, 10) == 0
        @info "Epoch " * string(epoch) * " done."
    end
end

## 6. Test on the witheld data

Prepare test data and test for accuracy. Use the cross-validated approach below for an average across subsamples. 

In [ ]:
test1 = [cats[:,61:end] dogs[:,61:end]]
StatsBase.transform!(dt, test1)
y_test = Φ * test1
labels_test = Int.(vcat(ones(20), zeros(20)))
mean(onecold(sigmoid.(m(y_test)), [0,1]) .== labels_test)

## 7. Cross-Validated Results

Here, we use simple random subsampling cross-validation to obtain an averaged result that mitigates the effect of the small sample size.  You should achieve around 80% accuracy or higher (although not guaranteed as already noted), which is quite remarkable in light of the fact that only 20 pixels were used on just 120 crudely-preprocessed images for training.

In [ ]:
cats = Float32.(readdlm("SSPOC/data/cats.csv", ','))
dogs = Float32.(readdlm("SSPOC/data/dogs.csv", ','))

function run_demo(cats, dogs, samples, rank, pixels)
    @info "Starting cross-validation on " * string(samples) * " samples..."
    out=zeros(samples)
    num_err = 0
    for i = 1:samples
        @label start
        cats_copy = deepcopy(cats)[:, sample(1:size(cats, 2), size(cats, 2), replace=false)];
        dogs_copy = deepcopy(dogs)[:, sample(1:size(dogs, 2), size(dogs, 2), replace=false)];

        # create training and testing sets (.74, .25)
        train1 = [cats_copy[:,1:60] dogs_copy[:,1:60]];
        test1 = [cats_copy[:,61:end] dogs_copy[:,61:end]];

        # compute PCA and project data to get eigenrepresentation X̃
        mean_tr = mean(train1, dims=2);
        train_centered = train1 - mean_tr * ones(1, size(train1, 2));
        U, S, V = svd(train_centered)
        Ψ = U[:,1:rank];
        X̃ = Ψ' * train1;

        # compute maximally discriminating component using LDA
        ## between-classes scatter Sb
        μ1 = mean(X̃[:, 1:60], dims=2);
        μ2 = mean(X̃[:,61:end], dims=2);
        Sb = (μ2 - μ1) * (μ2 - μ1)';

        function scatter_matrix(data, mu)
            (data - mu .* ones(size(data))) * (data - mu .* ones(size(data)))'
        end
        ## within-classes scatter Sw
        Σ1 = scatter_matrix(X̃[:,1:60], μ1);
        Σ2 = scatter_matrix(X̃[:,61:end], μ2);
        Sw = Σ1 + Σ2;

        # generalized eigenvalue decomposition to get projection matrix W
        Λ, W = eigen(Sb, Sw);
        ## choose eigenvector corresponding to largest eigenvalue w (this represents the maximally discriminating projection vector in the new subspace)
        Λ = reverse(real.(Λ));
        W = reverse(W, dims=2);
        w = real(W[:,1])

        # solve for S (this is the maximally sparse vector in original space that when projected to subspace approximates w)
        n = size(train1, 1)
        s = Variable(n)
        objective = norm(s,1)
        constraint = Ψ' * s == w;
        solver = COSMO.Optimizer;
        problem = minimize(objective, constraint);
        solve!(problem, solver, silent_solver=true)
        if isnothing(s.value)
            @warn "Encountered a numerical error during optimization!  Repeating sample..."
            num_err += 1
            @goto start
        end
        ŝ = s.value[:] 
        thresh = abs(ŝ[reverse(sortperm(abs.(ŝ)))[pixels + 1]])
        C = abs.(ŝ) .> thresh;
        
        # create measurement matrix
        Φ = Diagonal(C);

        # scale data (z score transform)
        dt = StatsBase.fit(ZScoreTransform, train1, dims=2);
        StatsBase.transform!(dt, train1);

        # sparsify original data with measurement matrix
        y = Φ * train1;

        # create labels
        labels = Int.(vcat(ones(60), zeros(60)))
        l = onehotbatch(labels, [0, 1]);

        # prepare data for NN
        train = DataLoader((y, l));

        # build logistic regression model with L1 regularization

        function loss(x, y)
            logitbinarycrossentropy(m(x), y)
        end

        # optimizer
        opt = ADADelta(.5)
        #opt = ADAM(1e-3)
        m = Dense(4096, 2)
        # train for 100 epochs
        for epoch in 1:100
            Flux.train!(loss, params(m), train, opt)
        end

        # prepare test data
        StatsBase.transform!(dt, test1);
        y_test = Φ * test1;
        labels_test = Int.(vcat(ones(20), zeros(20)));

        out[i] = mean(onecold(sigmoid.(m(y_test)), [0,1]) .== labels_test)
        @info "Sample "*string(i)*" accuracy: "*string(out[i])
    end
    out_text = num_err == 1 ? " sample" : " samples"
    @info string(num_err) * out_text * " repeated due to numerical errors."
return "Mean accuracy: " * string(sum(out)/(samples))
end
run_demo(cats, dogs, 10, 40, 20)


<span id="fn1">B. W. Brunton, S. L. Brunton, J. L. Proctor, J. N. Kutz. Sparse sensor placement optimization for classification.  *SIAM Journal on Applied Mathematics*, 76(5):2099-2122, 2016.</span>